In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
import seaborn as sns
import re
import requests

%matplotlib inline

In [2]:
papers = pd.read_csv(os.getcwd().replace('scripts/old', 'data/papers') + '/final_valid_papers.csv')

In [50]:
kwords = pd.read_csv(os.getcwd().replace('scripts/old', 'data')+'/technical_buzzwords.csv')
words = pd.DataFrame(columns = ['topic', 'keywords'])
for i in range(kwords.shape[0]):
    words.loc[len(words.index)] = [kwords.iat[i,0], str(kwords.iat[i,1]).split(", ")]



In [4]:
kwords

,buzzword,terms
0,AI and Machine Learning,"Artificial Intelligence, Machine Learning, Sup..."
1,Big Data,"Hadoop, NoSQL, Apache Flink, Apache Kafka, Big..."
2,Blockchain and Cryptocurrency,"Blockchain, Cryptocurrency, Bitcoin, Ethereum,..."
3,IoT,"Smart Medical Device, Industrial IoT, Home Aut..."
4,AR and VR,"Augmented Reality, Virtual Reality, Mixed Real..."
5,Cybersecurity,"Cybersecurity, Ethical Hacking, Intrusion Dete..."
6,Automation,"Automation, Industrial Automation, Home Automa..."
7,Modeling,"Modeling, Modelling, ArcGIS, HEC-RAS, MIKE, FL..."
8,NLP,"NLP, Natural Language Processing, Sentiment An..."
9,Quantum Computing,"Quantum Computing, Quantum Algorithm, Quantum ..."


In [5]:
keys = [
    "79092851a8414bbbde34f25e407cfc6b",
"df641c372194510cd18b35a01e032f50",
"f8df64a8c1f69df110dab9b4b9db68fe",
"492c6241cd068bd6fccbc3c08979995b",
"6cf520126463cddc241f37d6b19f1b51",
"942c18864635041e8f03a1181d82cc5b",
"a51dbedb4a586ee070f8b65376e3f526",
"de4e0b640b29a882d856610dda198778",
"6f16c7b10db8f2b886ff9a8083960750"
]

In [20]:
# ret = []

In [13]:
def find_keywords(k_list, abstract):
    ret = [] # all instances of found buzzwords
    
    old_abstract = abstract
    
    orig = old_abstract
    
    #loop thru all buzzwords
    for word in k_list:
        word = word.lower().replace('-', ' ')
        curr_abstract = old_abstract
        c_orig = orig
        done = False
        cnt = 0
        while(True):
            if done:
                break
            try:
                #try to find word
                found_i = curr_abstract.index(word)

                #find context of 100 characters around it
                l_b = max(0, found_i - 100)
                u_b = min(len(curr_abstract), found_i+len(word)+100)
                
                if(l_b > u_b):
                    break
                else:
                    cnt+=1

                if u_b >= len(curr_abstract):
                    break

                #shorten the text to take out the found keyword
                
                curr_abstract = curr_abstract[found_i + len(word):]
                c_orig = c_orig[found_i + len(word):]

            except ValueError:
                # print("DONE")
                #if cannot find then we are done
                done = True
                continue
            except Exception as e:
                break
        if cnt != 0:
            #append number of topics found and the specific keyword
            ret.append([word,cnt])

            # print([word, cnt])
    
    return ret

In [45]:
def cleaning(tmp):
#     print(tmp)
    #remove non-ascii
#     tmp = unicodedata.normalize('NFKD', tmp).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #remove URLs
#     tmp = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str)
    #remove punctuations
    tmp = re.sub(r'[^\w]|_',' ',tmp)
    #to lowercase
    tmp = tmp.lower()
    #Remove additional white spaces
    tmp = re.sub('[\s]+', ' ', tmp)

    return tmp
       

In [7]:
keyi = 0

In [72]:
#get the full text
def fullText(i, keyi):
    doi = papers.iat[i,4]
#     done = False
#     print(doi)
    try:
        js = requests.get(
            f"https://api.elsevier.com/content/article/doi/{doi}",
            headers = {"X-ELS-APIKey":keys[keyi], "Accept":"application/json"}
,
        )
        
        r = js.json()

        full_text = r['full-text-retrieval-response']['originalText']
#         full_text = re.sub(r'^https?:\/\/.[^\s]+', '', full_text)
#         print(full_text)
        
        abstr = r['full-text-retrieval-response']['coredata']['dc:description'][:25]
        
        index = full_text.find(abstr)
        if(index==-1):
            other.append(i)
            return None
        full_text = full_text[index:]
        
        full_text = ''.join(full_text.split('References')[:-1])

        full_text = cleaning(full_text)    
#         print(full_text)

        keyi+=1
        
        return full_text
    except Exception as e:
        if js.status_code == 404:
            error_df.loc[len(error_df.index)] = [i,doi]
        elif js.status_code ==429:
            print(str(e))
            keyi+=1
            return fullText(i, (keyi)%9)
        

        else:
            error_df.loc[len(error_df.index)] = [i,doi]
        return None
    


In [51]:
buzzwords_found = pd.DataFrame(columns = ["index", "doi", "buzz_id","category", "subword", "count"])
contexts = pd.DataFrame(columns = ["index", "doi", "context"])
error_df = pd.DataFrame(columns = ["index", "doi"])
full_text_df = pd.DataFrame(columns = ["index", "text"])

In [71]:
error_df_i = 8
for i in range(9000,papers.shape[0]):
    print(i)
    if i%1500==0 and i !=0:
        full_text_df.to_csv('/Volumes/mac/corpus/text_corpus'+str(error_df_i)+'.csv', index = False)
        error_df_i+=1
        full_text_df = pd.DataFrame(columns = ["index", "text"])

    # print(i)
    ft = fullText(i = i, keyi = keyi)
    
    if ft == None:
        error_df.loc[len(error_df.index)] = [i,papers.iat[i,4]]
    else:
        full_text_df.loc[len(full_text_df.index)] = [i, ft]
#     print(ft)
    for j in range(words.shape[0]):
        subterms = words.iat[j, 1]
#         print(subterms)
        for found in find_keywords(subterms, ft):
            buzzwords_found.loc[len(buzzwords_found.index)] = [i,papers.iat[i,4], j,words.iat[j,0], found[0], found[1]]


9000
9001
9002
9003
9004
9005
9006
9007
9008
9009
9010
9011
9012
9013
9014
9015
9016
9017
9018
9019
9020
9021
9022
9023
9024
9025
9026
9027
9028
9029
9030
9031
9032
9033
9034
9035
9036
9037
9038
9039
9040
9041
9042
9043
9044
9045
9046
9047
9048
9049
9050
9051
9052
9053
9054
9055
9056
9057
9058
9059
9060
9061
9062
9063
9064
9065
9066
9067
9068
9069


KeyboardInterrupt: 

In [66]:
buzzwords_found[buzzwords_found['index']==8075]

,index,doi,buzz_id,category,subword,count
49312,8075,10.1016/j.ejrh.2022.100988,0,AI and Machine Learning,classification,1
49313,8075,10.1016/j.ejrh.2022.100988,0,AI and Machine Learning,support vector machine,1
49314,8075,10.1016/j.ejrh.2022.100988,0,AI and Machine Learning,regression,15
49315,8075,10.1016/j.ejrh.2022.100988,7,Modeling,arcgis,1
49316,8075,10.1016/j.ejrh.2022.100988,7,Modeling,mike,1
49317,8075,10.1016/j.ejrh.2022.100988,7,Modeling,3d model,5
49318,8075,10.1016/j.ejrh.2022.100988,7,Modeling,simulation,50
49319,8075,10.1016/j.ejrh.2022.100988,7,Modeling,model prediction,1
49320,8075,10.1016/j.ejrh.2022.100988,7,Modeling,model parameters,3
49321,8075,10.1016/j.ejrh.2022.100988,7,Modeling,model development,1


In [70]:
full_text_df = full_text_df.loc[1152:]

In [74]:
full_text_df.to_csv('full_text_df.csv', index = False)

In [33]:
word_contexts.to_csv(os.getcwd().replace('scripts', 'data')+'/word_contexts.csv', index = False)




In [47]:
no_data

[]

In [39]:

pd.DataFrame(columns = ['index'], data = other).to_csv(os.getcwd().replace('scripts', 'data')+'/full_text_other_error.csv', index = False)



In [29]:
word_contexts = word_contexts.loc[:696970].drop('context', axis = 1)

In [32]:
word_contexts

,paper_id,category,sub_word
0,0,Modeling,modeling
1,0,Modeling,modeling
2,0,Modeling,modeling
3,0,Modeling,modeling
4,0,Modeling,modeling
...,...,...,...
1111686,27399,Infrastructure as a Service (IaaS),databases
1111687,27399,Infrastructure as a Service (IaaS),databases
1111688,27399,Infrastructure as a Service (IaaS),databases
1111689,27399,Infrastructure as a Service (IaaS),databases


In [23]:
contexts = word_contexts['context'].tolist()

In [35]:
pd.DataFrame(columns = ['context'], data = contexts).to_csv(os.getcwd().replace('scripts', 'data')+'/paper_contexts.csv', index = False)

